In [1]:
cd ..

/home/sumgh/Data Science/projects/Chat_with_pdf


In [2]:
from src.helper import *

/home/sumgh/Data Science/projects/Chat_with_pdf/pdf_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from params import *

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.environ.get('GOOGLE_API_KEY')

In [4]:
documents = get_documnts_from_pdf("/home/sumgh/Documents")

In [5]:
database_path = "artifacts/fassi_index"

In [6]:
text_chunks = text_splitter(documents, chunk_size=1000, chunk_overlap=200)
create_knowledgebase(texts=text_chunks, db_path=database_path)

In [4]:
### Prompt
template="""You are an assistant for question-answering tasks.
                Use the following pieces of retrieved context to answer the question.
                If you don't know the answer, just say that you don't know.
                Use five sentences maximum and keep the answer concise.
                Question: {question}
                Context: {context}
                Chat history: {chat_history}
                Answer:    
            """

prompt=ChatPromptTemplate.from_template(template)

In [5]:
### retriever
vectordb = FAISS.load_local(folder_path="artifacts/fassi_index", embeddings=load_embedding(), allow_dangerous_deserialization=True)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True)

In [7]:
chat_history = []

In [8]:
def input_handeler(input: dict):
    return input['question']

In [9]:
### Chain
rag_chain = (
            RunnablePassthrough.assign(
                context = input_handeler | retriever
            )
            | prompt
            | llm
            | StrOutputParser()
        )

In [10]:
from langchain.schema import AIMessage, HumanMessage

In [11]:
question = "What is my previous question?"
# responce = rag_chain.invoke({"question": question, "chat_history":chat_history})
responce = rag_chain.stream({"question": question, "chat_history":chat_history})

In [13]:
for smg in responce:
    print(smg)


I
 don
't
 know
 the
 answer
 as
 the
 previous
 question
 is
 not
 provided
 in
 the
 retrieved
 context
.



In [15]:
responce.()

TypeError: throw expected at least 1 argument, got 0

In [30]:
chat_history.extend(
    [
        HumanMessage(
            role="user", 
            content=question
        ),
        AIMessage(
            role="assistant",
            content=responce
        ),
    ]  
)

In [31]:
for chat in chat_history:
    print(chat)

content='What is my previous question?' role='user'
content="I don't know the answer as the previous question is not provided in the retrieved context." role='assistant'
content='What is my previous question?' role='user'
content="I don't know the answer as the previous question is not provided in the retrieved context." role='assistant'


In [27]:
# chat_history.extend(
#     [
#         HumanMessage(
#             role="user", 
#             content=question
#         ),
#         AIMessage(
#             role="assistant",
#             content=responce
#         ),
#     ]    
# )

In [28]:
chat_history

[HumanMessage(content='What is self attention?', role='user'),
 AIMessage(content='Self-attention is an attention mechanism that relates different positions of a single sequence to compute a representation of the sequence. It has been used successfully in various tasks like reading comprehension, abstractive summarization, and learning task-independent sentence representations. The Transformer model is the first transduction model that relies entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution. Self-attention allows the model to draw global dependencies between input and output, enabling more parallelization and achieving state-of-the-art translation quality. In self-attention, all positions are connected with a constant number of sequentially executed operations, making it faster than recurrent layers for smaller sequence lengths. Self-attention can be restricted to considering only a neighborhood of a certai